In [28]:
import pandas as pd
import plotly.io as pio
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pio.templates.default = "plotly_dark"

In [113]:
baseURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/"
def loadData(fileName, columnName):
    data = pd.read_csv(baseURL + fileName) \
             .drop(['Lat', 'Long'], axis=1) \
             .melt(id_vars=['Province/State', 'Country/Region'], 
                 var_name='date', value_name=columnName) \
             .astype({'date':'datetime64[ns]', columnName:'Int64'}, 
                 errors='ignore')
    data['Province/State'].fillna('<all>', inplace=True)
    data[columnName].fillna(0, inplace=True)
    return data

In [164]:
allData = loadData(
    "time_series_covid19_confirmed_global.csv", "CumConfirmed") \
  .merge(loadData(
    "time_series_covid19_deaths_global.csv", "CumDeaths")) \
  .merge(loadData(
    "time_series_covid19_recovered_global.csv", "CumRecovered"))

In [165]:
countries = allData['Country/Region'].unique()
countries.sort()

In [166]:
allData = allData.groupby(['Country/Region', 'date']).sum().reset_index().set_index('date')

In [195]:
# allData = allData.groupby(['Country/Region']) 
country_filter = allData.groupby(['Country/Region']).max().CumConfirmed > 10000
country_filter = country_filter.index[country_filter].values

In [196]:
allData_filtered = allData[allData['Country/Region'].isin(country_filter)]

In [218]:
allData_filtered

,Country/Region,CumConfirmed,CumDeaths,CumRecovered
date,,,,
2020-01-22,Austria,0,0,0
2020-01-23,Austria,0,0,0
2020-01-24,Austria,0,0,0
2020-01-25,Austria,0,0,0
2020-01-26,Austria,0,0,0
...,...,...,...,...
2020-04-04,United Kingdom,42477,4320,215
2020-04-05,United Kingdom,48436,4943,229
2020-04-06,United Kingdom,52279,5385,287


In [197]:
px.line(allData_filtered, x=allData_filtered.index, y='CumDeaths', color='Country/Region', width=1200, height=600)
# px.line(df2, x=df2.index, y=df2.values[:, 1])
# px.line(df2, x=df2.index, y=df2.values[:, 2])

In [198]:
px.line(allData_filtered, x=allData_filtered.index, y='CumConfirmed', color='Country/Region', width=1200, height=600)

In [199]:
px.line(allData_filtered, x=allData_filtered.index, y='CumRecovered', color='Country/Region', width=1200, height=600)

In [220]:
px.pie(allData_filtered, values='CumRecovered', names='Country/Region', width=1200, height=600, color_discrete_sequence=px.colors.sequential.RdBu)